#### Example of using crosswalks between census tracts of different years

This notebook used to provide example code for using the output from the script in creating geojsons with multiple years of census data for the same tract. As I'm migrating it to a dedicated script in main.py, it's now for testing different parts of that script.  

#### 1.) Download the crosswalks from the Azure container 

In [3]:
import utils

In [1]:
from tract_crosswalk import get_tract_crosswalks

get_tract_crosswalks()

2023-12-14 09:30:15,548 - logger - INFO - (OVERWRITE_AZURE=False, OVERWRITE_LOCAL=False)
2023-12-14 09:30:15,710 - logger - INFO - Checking if crosswalks exist:
{'file': 'convert-ctracts_pct-area_2010-to-2020.json', 'local': True, 'azure': True}
{'file': 'convert-ctracts_pct-area_2020-to-2010.json', 'local': True, 'azure': True}
2023-12-14 09:30:15,711 - logger - INFO - Local crosswalks exist and OVERWRITE_LOCAL=False. Aborting script.


In [4]:
map_10_to_20 = utils.read_json_rows('convert-ctracts_pct-area_2010-to-2020.json')



FileNotFoundError: [Errno 2] No such file or directory: 'convert-ctracts_pct-area_2010-to-2020.json'

In [4]:
from tract_crosswalk import load_crosswalk_2010_2020
import pandas as pd

map_10_to_20 = load_crosswalk_2010_2020()

In [5]:
df = pd.DataFrame(map_10_to_20)

In [6]:
df

,STATENAME,GEOID_TRACT_10,GEOID_TRACT_20_overlap
0,Alabama,01001020100,"{'01001020100': 0.999, '01001020803': 0.0}"
1,Alabama,01001020200,"{'01001020100': 0.0, '01001020200': 1.0}"
2,Alabama,01001020300,{'01001020300': 1.0}
3,Alabama,01001020400,{'01001020400': 1.0}
4,Alabama,01001020500,"{'01001020501': 1.0, '01001020502': 1.0, '0100..."
...,...,...,...
74353,Wyoming,56043000200,"{'56043000200': 1.0, '56043000302': 0.008}"
74354,Wyoming,56043000301,{'56043000301': 1.0}
74355,Wyoming,56043000302,"{'56043000200': 0.0, '56043000302': 0.992}"
74356,Wyoming,56045951100,"{'56045951100': 1.0, '56045951300': 0.0}"


In [1]:
import utils 

utils.construct_raw_census_output_path()

'data/raw/B01001A_012E-B01001A_017E-B19013_001E_allStates+DC_2010-2020.csv'

In [2]:
from census_data import download_raw_data

df = download_raw_data()

2023-12-14 11:19:44,260 - logger - INFO - Downloading ['B01001A_012E', 'B01001A_017E', 'B19013_001E'] from 2010 to 2020: data/raw/B01001A_012E-B01001A_017E-B19013_001E_allStates+DC_2010-2020.csv
2023-12-14 11:19:45,723 - logger - INFO - Downloaded (AL, 2010, 1s, 1181 rows, 0.5581798553466797MB)
2023-12-14 11:19:45,724 - logger - INFO - NAME, B01001A_012E, B01001A_017E, B19013_001E, state, county, tract, year, state_fips, state_name, state_usps

2023-12-14 11:19:46,988 - logger - INFO - Downloaded (AL, 2011, 1s, 1181 rows, 0.5581798553466797MB)
2023-12-14 11:19:46,989 - logger - INFO - NAME, B01001A_012E, B01001A_017E, B19013_001E, state, county, tract, year, state_fips, state_name, state_usps

2023-12-14 11:19:48,354 - logger - INFO - Downloaded (AL, 2012, 1s, 1181 rows, 0.5581798553466797MB)
2023-12-14 11:19:48,355 - logger - INFO - NAME, B01001A_012E, B01001A_017E, B19013_001E, state, county, tract, year, state_fips, state_name, state_usps

2023-12-14 11:19:49,773 - logger - INFO - D

In [1]:
import pandas as pd 
import utils
import census_data
import yaml
from tract_crosswalk import load_crosswalk_2010_2020

# census_data.download_raw_data()

with open('config.yaml', 'r') as file: 
    config = yaml.full_load(file)

raw_census_table_path = utils.construct_raw_census_output_path()
raw_census_table = pd.read_csv(raw_census_table_path, index_col=0, dtype={'state_fips':str})

raw_census_table
df = raw_census_table.copy()

import census_data
long_df = census_data._transform_raw_data_long(df)
wide_df = census_data._widen_df(long_df)
df_2020 = census_data._extract_2020_data(wide_df)
collapsed_df = census_data._collapse_df(wide_df)

# Load crosswalk from 2010 ot 2020
df = collapsed_df.copy()
df_map_10_to_20 = pd.DataFrame(load_crosswalk_2010_2020(reverse=False))

joined = census_data.join_crosswalk(df)
# joined = joined.dropna()

df_map_10_to_20 = pd.DataFrame(load_crosswalk_2010_2020(reverse=False))

2023-12-14 14:28:47,930 - logger - INFO - Raw Census data in data/raw/B19013_001E-B01001A_017E_DE-MD-NJ-PA-RI+DC_2010-2020.csv:
NAME, B19013_001E, B01001A_017E, state, county, tract, year, state_usps
"Census Tract 169.04,  New Castle County,  Delaware", 70875.0, 668.0, 10, 3, 16904, 2010, DE
"Census Tract 9801,  New Castle County,  Delaware", -666666666.0, 0.0, 10, 3, 980100, 2010, DE
"Census Tract 9901,  New Castle County,  Delaware", -666666666.0, 0.0, 10, 3, 990100, 2010, DE
"Census Tract 501.01,  Sussex County,  Delaware", 52569.0, 1510.0, 10, 5, 50101, 2010, DE
(78574, 8)

2023-12-14 14:28:48,342 - logger - INFO - 934 rows have negative placeholder values for census variables:
 GEOID, state_fips, state_name, state_usps, county_fips, county_name, tract_fips, tract_dec, year, B19013_001E, B01001A_017E, count
10001980000, 10, Delaware, DE, 001, Kent, 980000, 9800, 2020, -666666666.0, 48.0, 1
42019910500, 42, Pennsylvania, PA, 019, Butler, 910500, 9105, 2012, -666666666.0, 696.0, 1
42

In [2]:
dataframes = []
for cvar in config['census_vars']:
    display(joined[cvar])
    var_df = pd.DataFrame(census_data.multiply_pct_overlaps(joined[cvar], joined['GEOID_TRACT_20_overlap'])).T.rename_axis('GEOID_TRACT_20')
    var_df.columns = [cvar + "-" + str(col) for col in var_df.columns]
    
    dataframes.append(var_df)
df2= pd.concat(dataframes, axis=1)

df2

GEOID        state_fips  state_name    state_usps  county_fips  county_name  tract_fips  tract_dec
10001040100  10          Delaware      DE          001          Kent         040100      401          {'2010': 63292.0, '2011': 70055.0, '2012': 742...
10001040201  10          Delaware      DE          001          Kent         040201      402.01       {'2010': 52367.0, '2011': 54770.0, '2012': 492...
10001040202  10          Delaware      DE          001          Kent         040202      402.02       {'2010': 62210.0, '2011': 67655.0, '2012': 741...
10001040203  10          Delaware      DE          001          Kent         040203      402.03       {'2010': 50000.0, '2011': 54124.0, '2012': 556...
10001040204  10          Delaware      DE          001          Kent         040204      402.04                                                      {}
                                                                                                                            ...              

2023-12-14 14:28:51,633 - logger - DEBUG - {'2010': 63292.0, '2011': 70055.0, '2012': 74211.0, '2013': 71141.0, '2014': 61270.0, '2015': 59094.0, '2016': 57375.0, '2017': 63324.0, '2018': 63826.0, '2019': 75194.0}
2023-12-14 14:28:51,634 - logger - DEBUG - {'2010': 52367.0, '2011': 54770.0, '2012': 49288.0, '2013': 52857.0, '2014': 51103.0, '2015': 51423.0, '2016': 50744.0, '2017': 50202.0, '2018': 52065.0, '2019': 54800.0}
2023-12-14 14:28:51,634 - logger - DEBUG - {'2010': 62210.0, '2011': 67655.0, '2012': 74167.0, '2013': 66998.0, '2014': 68947.0, '2015': 70877.0, '2016': 68205.0, '2017': 66243.0, '2018': 71384.0, '2019': 74146.0}
2023-12-14 14:28:51,635 - logger - DEBUG - {'2010': 50000.0, '2011': 54124.0, '2012': 55648.0, '2013': 57075.0, '2014': 68169.0, '2015': 62209.0, '2016': 62708.0, '2017': 64621.0, '2018': 67159.0, '2019': 66797.0}
2023-12-14 14:28:51,635 - logger - DEBUG - {}


AttributeError: 'float' object has no attribute 'items'